In [5]:
#Importing Dependencies
!pip install tensorflow tensorflow-gpu opencv-python mediapipe sklearn matplotlib 

You should consider upgrading via the 'c:\users\harsh\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [65]:
import cv2 
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from cv2 import VideoCapture

In [66]:
mp_holistic = mp.solutions.holistic  #Holistic
mp_drawing = mp.solutions.drawing_utils # Drawing Utitlities

In [67]:
def mediapipe_detection(image, model) :
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #Color conversion
    image.flags.writeable = False                  #Image is no longer writable 
    results = model.process(image)                 #make prediction
    image.flags.writeable = True                   #Image is writable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) #Color conversion back
    return image, results

In [68]:
def draw_landmarks(image, results) : 
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [69]:
def draw_styled_landmarks(image, results) : 
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

In [6]:
#Access the webcam
cap = VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic : 
    while cap.isOpened() :

        #Reading frames
        ret, frame = cap.read()

        #Make detection 
        image, results = mediapipe_detection(frame, holistic)
        draw_styled_landmarks(image, results)
        #print(results)

        #show to screen
        cv2.imshow('OpenCV Feed', image)

        #break 
        if cv2.waitKey(10) & 0xFF == ord('q') : 
            break

    cap.release()
    cv2.destroyAllWindows()

In [17]:
np.array(results.pose_landmarks.landmark).shape

(33,)

In [19]:
np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().shape

(132,)

In [70]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132*4)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [71]:
def extract_keypoints(results) : 
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [71]:
extract_keypoints(results).shape

(1662,)

In [10]:
results.face_landmarks

landmark {
  x: 0.5606679916381836
  y: 0.6539683938026428
  z: -0.032830365002155304
}
landmark {
  x: 0.5662314891815186
  y: 0.5916970372200012
  z: -0.07353311777114868
}
landmark {
  x: 0.5623826384544373
  y: 0.6100836992263794
  z: -0.03654862567782402
}
landmark {
  x: 0.5544643998146057
  y: 0.5201610922813416
  z: -0.06093516945838928
}
landmark {
  x: 0.5674204230308533
  y: 0.5701866149902344
  z: -0.07946225255727768
}
landmark {
  x: 0.5675632953643799
  y: 0.5414620041847229
  z: -0.07589861750602722
}
landmark {
  x: 0.566236674785614
  y: 0.47202572226524353
  z: -0.04312750697135925
}
landmark {
  x: 0.4601966142654419
  y: 0.4617408215999603
  z: 0.0005335337482392788
}
landmark {
  x: 0.5669883489608765
  y: 0.41925328969955444
  z: -0.03730640932917595
}
landmark {
  x: 0.5683510303497314
  y: 0.3900718688964844
  z: -0.042235732078552246
}
landmark {
  x: 0.5701151490211487
  y: 0.289712518453598
  z: -0.036063361912965775
}
landmark {
  x: 0.5602093935012817
  y:

In [72]:
#path for exported array
DATA_PATH = os.path.join('MP_Data')

#Actions that we are going to detect
actions = np.array(['hello', 'thanks', 'iloveyou'])

# 30 videos worth of data
no_sequences = 30 

#30 frames of length
sequence_length = 30 

In [73]:
for action in actions : 
    for sequence in range(no_sequences) : 
        try : 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except : 
            pass

In [93]:
cap = VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic : 
    #Loop through actions 
    for action in actions :
        #Loop through the collection of frames
        for sequence in range(no_sequences) : 
            
            #Loop through each frame
            for frame_num in range(sequence_length) : 
                
                #Reading frames
                ret, frame = cap.read()

                #Make detection 
                image, results = mediapipe_detection(frame, holistic)
                draw_styled_landmarks(image, results)
                #print(results)
                
                
                #collecting images 
                
                if frame_num == 0 : 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 1, cv2.LINE_AA)
                    cv2.putText(image, f'Collecting frames for {action} video number {sequence}', (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 4, cv2.LINE_AA)
                    cv2.waitKey(2000)
                else : 
                    cv2.putText(image, f'Collecting frames for {action} video number {sequence}', (15,12), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 4, cv2.LINE_AA)
                
                #show to screen
                cv2.imshow('OpenCV Feed', image)
                
                #Saving the arrays
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                #break 
                if cv2.waitKey(10) & 0xFF == ord('q') : 
                    break

    cap.release()
    cv2.destroyAllWindows()

In [73]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [74]:
label_map = {label : num for num, label in enumerate(actions)}

In [75]:
label_map

{'hello': 0, 'thanks': 1, 'iloveyou': 2}

In [76]:
#combining all sequences 

sequences = []
labels = []
for action in actions : 
    for sequence in range(no_sequences) : 
        window = []
        for frame_num in range(sequence_length) : 
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), f'{str(frame_num)}.npy'))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [130]:
np.array(sequences).shape

(90, 30, 1662)

In [77]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)

In [132]:
y

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0,

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05)

In [78]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [79]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [85]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation= 'relu', input_shape = (30,1662)))
model.add(LSTM(128, return_sequences=True, activation = 'relu'))
model.add(LSTM(64, return_sequences=False, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(actions.shape[0], activation = 'softmax'))

In [86]:
model.compile(optimizer= 'Adam', loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])

In [41]:
model.fit(X_train, y_train , epochs = 245, callbacks=[tb_callback])

Epoch 1/245
3/3 [==============================] - 7s 151ms/step - loss: 4.2360 - categorical_accuracy: 0.8471
Epoch 2/245
3/3 [==============================] - 0s 115ms/step - loss: 5.5776 - categorical_accuracy: 0.5059
Epoch 3/245
3/3 [==============================] - 0s 102ms/step - loss: 1.3993 - categorical_accuracy: 0.6000
Epoch 4/245
3/3 [==============================] - 1s 183ms/step - loss: 0.4668 - categorical_accuracy: 0.8471
Epoch 5/245
3/3 [==============================] - 0s 140ms/step - loss: 0.5325 - categorical_accuracy: 0.8118
Epoch 6/245
3/3 [==============================] - 0s 115ms/step - loss: 0.3355 - categorical_accuracy: 0.9412
Epoch 7/245
3/3 [==============================] - 0s 128ms/step - loss: 0.2463 - categorical_accuracy: 0.9412
Epoch 8/245
3/3 [==============================] - 0s 87ms/step - loss: 0.2274 - categorical_accuracy: 0.9412
Epoch 9/245
3/3 [==============================] - 0s 141ms/step - loss: 0.1875 - categorical_accuracy: 0.9294
Ep

3/3 [==============================] - 0s 119ms/step - loss: 0.0966 - categorical_accuracy: 0.9647
Epoch 75/245
3/3 [==============================] - 0s 95ms/step - loss: 0.1256 - categorical_accuracy: 0.9647
Epoch 76/245
3/3 [==============================] - 0s 107ms/step - loss: 4.6166 - categorical_accuracy: 0.6588
Epoch 77/245
3/3 [==============================] - 0s 110ms/step - loss: 5.0999 - categorical_accuracy: 0.3529
Epoch 78/245
3/3 [==============================] - 0s 116ms/step - loss: 2.2620 - categorical_accuracy: 0.3765
Epoch 79/245
3/3 [==============================] - 0s 95ms/step - loss: 1.4092 - categorical_accuracy: 0.3647
Epoch 80/245
3/3 [==============================] - 0s 110ms/step - loss: 0.8129 - categorical_accuracy: 0.6118
Epoch 81/245
3/3 [==============================] - 0s 106ms/step - loss: 0.7032 - categorical_accuracy: 0.7294
Epoch 82/245
3/3 [==============================] - 0s 93ms/step - loss: 0.6215 - categorical_accuracy: 0.7882
Epoch 83

3/3 [==============================] - 0s 110ms/step - loss: 0.0421 - categorical_accuracy: 0.9882
Epoch 148/245
3/3 [==============================] - 0s 112ms/step - loss: 0.0966 - categorical_accuracy: 0.9647
Epoch 149/245
3/3 [==============================] - 0s 96ms/step - loss: 0.0604 - categorical_accuracy: 0.9647
Epoch 150/245
3/3 [==============================] - 0s 116ms/step - loss: 0.0923 - categorical_accuracy: 0.9647
Epoch 151/245
3/3 [==============================] - 0s 135ms/step - loss: 0.1298 - categorical_accuracy: 0.9412
Epoch 152/245
3/3 [==============================] - 0s 111ms/step - loss: 0.0738 - categorical_accuracy: 0.9765
Epoch 153/245
3/3 [==============================] - 0s 160ms/step - loss: 0.1845 - categorical_accuracy: 0.9176
Epoch 154/245
3/3 [==============================] - 0s 111ms/step - loss: 0.1181 - categorical_accuracy: 0.9529
Epoch 155/245
3/3 [==============================] - 0s 139ms/step - loss: 0.0892 - categorical_accuracy: 0.964

3/3 [==============================] - 0s 123ms/step - loss: 0.0673 - categorical_accuracy: 0.9765
Epoch 220/245
3/3 [==============================] - 0s 116ms/step - loss: 0.0257 - categorical_accuracy: 1.0000
Epoch 221/245
3/3 [==============================] - 0s 108ms/step - loss: 0.0641 - categorical_accuracy: 0.9765
Epoch 222/245
3/3 [==============================] - 0s 105ms/step - loss: 0.0951 - categorical_accuracy: 0.9647
Epoch 223/245
3/3 [==============================] - 0s 156ms/step - loss: 0.0481 - categorical_accuracy: 0.9882
Epoch 224/245
3/3 [==============================] - 0s 100ms/step - loss: 0.0841 - categorical_accuracy: 0.9765
Epoch 225/245
3/3 [==============================] - 0s 134ms/step - loss: 0.0401 - categorical_accuracy: 0.9882
Epoch 226/245
3/3 [==============================] - 0s 112ms/step - loss: 0.0792 - categorical_accuracy: 0.9765
Epoch 227/245
3/3 [==============================] - 0s 98ms/step - loss: 0.3420 - categorical_accuracy: 0.847

In [42]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            442112    
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-tr

In [88]:
res= model.predict(X_test)

1/1 [==============================] - 0s 313ms/step


In [89]:
res

array([[4.2240144e-12, 3.1813145e-09, 1.0000000e+00],
       [1.8118157e-12, 2.0148516e-09, 1.0000000e+00],
       [2.1127683e-19, 1.0000000e+00, 6.0739567e-19],
       [2.8947564e-14, 1.9281301e-09, 1.0000000e+00],
       [2.1091804e-08, 9.9999988e-01, 8.5216030e-08]], dtype=float32)

In [54]:
actions[np.argmax(res[2])]

'thanks'

In [55]:
actions[np.argmax(y_test[2])]

'thanks'

In [144]:
model.save('action.h5')

In [84]:
del model

In [87]:
model.load_weights('action.h5')

In [90]:
# res = []
sequence = []
sentence = []
threshold = 0.4

In [183]:
actions[np.argmax(res)]

'iloveyou'

In [93]:
cap = VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic : 
    while cap.isOpened() :

        #Reading frames
        ret, frame = cap.read()

        #Make detection 
        image, results = mediapipe_detection(frame, holistic)
        draw_styled_landmarks(image, results)
        #print(results)
        
        #determining the result
        keypoints = extract_keypoints(results)
        sequence.insert(0, keypoints)
        sequence = sequence[:30]
        
        if len(sequence) == 30 :
            print(np.array(sequence).shape)
            res= model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])

            #SHOWING THE RESULTS

            #checking if the value of the predicted action is greater than the threshold
            if res[np.argmax(res)] > threshold: 

                #checking if the sentence has more than word
                if len(sentence) > 0: 

                    #if the predicted word doesn't match the last word, add it
                    if actions[np.argmax(res)] != sentence[-1] : 
                        sentence.append(actions[np.argmax(res)])

                    #if the predicted word matches the last word 
                else : 
                    sentence.append(actions[np.argmax(res)])

        #if the length of the sentence is greater than 5
        if len(sentence) > 5 : 

            #reduce the array to the last five values
            sentence = sentence[-5:]

        cv2.rectangle(image, (0,0), (640,40), (245,117,16) , -1)
        cv2.putText(image, ' '.join(sentence) , (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1 , (255,255,255), 1,  cv2.LINE_AA)

        #show to screen
        cv2.imshow('OpenCV Feed', image)


        #break 
        if cv2.waitKey(10) & 0xFF == ord('q') : 
            break

    cap.release()
    cv2.destroyAllWindows()

(30, 1662)
1/1 [==============================] - 0s 57ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 63ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 16ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 49ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 50ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 63ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 46ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 64ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 65ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 50ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 49ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 54ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 41ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 54ms/step
hello
(30, 1662)
1/1 [==================

1/1 [==============================] - 0s 32ms/step
iloveyou
(30, 1662)
1/1 [==============================] - 0s 19ms/step
iloveyou
(30, 1662)
1/1 [==============================] - 0s 26ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 16ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 14ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 18ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 25ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 16ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 33ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 20ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 15ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 28ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 24ms/step
thanks
(30, 1662)
1/1 [==============================] - 0s 38ms/step
thanks
(30, 1662)
1/1 [===========

1/1 [==============================] - 0s 32ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 16ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 26ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 16ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 32ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 15ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 20ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 35ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 15ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 47ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 24ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 22ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 29ms/step
hello
(30, 1662)
1/1 [==============================] - 0s 20ms/step
hello
(30, 1662)
1/1 [=============================

In [92]:
cap.release()
cv2.destroyAllWindows()

c:\users\harsh\appdata\local\programs\python\python39\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


(30,)